### Implementation of CSWAP operation.

The CSWAP gate exchanges the second and third qubit when the first qubit (LSB) is |1>.

In [15]:
import numpy as np
from qiskit import QuantumCircuit, assemble, Aer,QuantumRegister, ClassicalRegister,execute, IBMQ
from qiskit.tools.monitor import job_monitor
sim = Aer.get_backend('aer_simulator')

In [16]:
q = QuantumRegister(3)
qc = QuantumCircuit(q)
qc.cswap(q[0],q[1],q[2])
qc.draw()

q21_0: ─■─
        │ 
q21_1: ─X─
        │ 
q21_2: ─X─

### Implementing the Quantum swap circuit

#### How is the inner product estimated?

Initially the state of the system is $|0>|\psi>|\phi>$, after passing the ancilla bit through the hadamard gate the state changes to $1/\sqrt(2)(|0> + |1>)|\psi>|\phi>$, then after the passing through the cswap gate the state becomes $1/\sqrt(2)(|0>|\psi>|\phi> + |1>|\phi>|\psi>)$, then the second hadamard gate results in 1/2|0⟩(|ϕ⟩|ψ⟩+|ψ⟩|ϕ⟩)+1/2|1⟩(|ϕ⟩|ψ⟩−|ψ⟩|ϕ⟩).
Then the measurement gate measures |1> with probability 
1/2(⟨ϕ|⟨ψ| - ⟨ψ|⟨ϕ|)1/2(|ϕ⟩|ψ⟩ - |ψ⟩|ϕ⟩) = 1/2 - 1/2$|<\psi|\phi>|^2$. From this we can extract the value of inner product.

In [20]:
qq = QuantumRegister(3,'qq')
c = ClassicalRegister(1,'c')

circuit = QuantumCircuit(qq,c)

circuit.h(qq[0]) #Applying a Hadamard gate
circuit.x(qq[1]) #Comment to make both states non-orthogonal
circuit.cswap(qq[0],qq[1],qq[2]) # Contolled SWAP gate
circuit.h(qq[0])
circuit.measure(qq[0],c[0]) #Measuring the qubit

print(circuit)

nShots = 10000
job = execute(circuit, sim, shots=nShots)

job_monitor(job)

counts = job.result().get_counts()

if '1' in counts:
    b = counts['1']
else:
    b = 0
    
s = 1-(2/nShots)*(b)

print("Squared Inner Product:",str(s))
print("Counts: ",counts)

      ┌───┐   ┌───┐┌─┐
qq_0: ┤ H ├─■─┤ H ├┤M├
      ├───┤ │ └───┘└╥┘
qq_1: ┤ X ├─X───────╫─
      └───┘ │       ║ 
qq_2: ──────X───────╫─
                    ║ 
 c: 1/══════════════╩═
                    0 
Job Status: job has successfully run
Squared Inner Product: -0.007200000000000095
Counts:  {'0': 4964, '1': 5036}
